## Convolutional Neural Networks
### Importar Librerías

In [1]:
import time
inicio = time.time()
import numpy as np
import os
import re
import shutil
import random
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from PIL import Image

In [2]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential,Model
from tensorflow.python.keras.models import Input
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.utils import plot_model
from skimage.transform import resize
#from torchvision.transforms import Resize
from tensorflow.keras.preprocessing.image import ImageDataGenerator



## Cargar set de Imágenes

### Separar imagenes para validacion

In [3]:
data_dir = 'imgs60X'
valid_dir = 'val60X'

In [4]:
def select_aleat_image(path):
    
    # Select random file
    #random_folder = random.choice(os.listdir(path))
    random_file = random.choice(
    os.listdir(os.path.join(path)))

    # Create image path
    image_path = os.path.join(path, random_file)
    image_path = os.path.normpath(image_path)

    return image_path

In [5]:
## Crear data temporal
ruta_path = os.getcwd() + os.sep
origen = ruta_path + 'data2'
destino = ruta_path + 'data2_temp'
shutil.copytree(origen, destino)

'/home/jorge/EjemplosPY/WoodTracer/Wood60X/data2_temp'

In [6]:
#dirname = os.path.join(os.getcwd(), 'data2_temp')
imgpath = destino + os.sep
ruta1 = ruta_path + data_dir + os.sep
ruta2 = ruta_path + valid_dir + os.sep
#dirname1 = os.path.join(os.getcwd(), 'val60X')
#imgpath1 = dirname1 + os.sep

print("leyendo imagenes de ",imgpath)

leyendo imagenes de  /home/jorge/EjemplosPY/WoodTracer/Wood60X/data2_temp/


In [7]:
for root, dirnames, filenames in os.walk(imgpath):
    for dirname in dirnames:
        path1 = root + dirname
        folder_nomber = path1.split('/')[7]
        path_dir = ruta2 + folder_nomber + os.sep
        os.mkdir(path_dir)
        for i in range(3):
            image_path = select_aleat_image(path1)
            name_image = image_path.split('/')[8]
            new_path = path_dir + dirname + '-' + name_image
            # Mover archivo
            os.rename(image_path, new_path)

# Funcion de Augmantation data
# tomar una imagen aleatoriamente y hacerle cambios, con "augmentation data" y guardar las imagenes
# esto hasta que la cantidad de cada categoria sea igual a 1000

origen1 = ruta_path + 'data2_temp'
destino1 = ruta_path + 'imgs60X'
shutil.copytree(origen1, destino1)
shutil.rmtree(origen1, ignore_errors=False)

In [8]:
dirname = os.path.join(os.getcwd(), 'imgs60X')
imgpath = dirname + os.sep 

images = []
directories = []
dircount = []
prevRoot=''
cant=0


print("leyendo imagenes de ",imgpath)

# Definir una función para cargar imágenes de una categoría
def cargar_imagenes_de_categoria(categoria):
    images = []
    cant=0
    
    for root, _, filenames in os.walk(os.path.join(data_dir, categoria)):
        for filename in filenames:
            if re.search("\.(jpg|jpeg|png|bmp|tiff)$", filename):
#                 cant=cant+1
                filepath = os.path.join(root, filename)
                image = plt.imread(filepath)
                image_resized = resize(image, (50, 50), anti_aliasing=True, clip=False, preserve_range=True)
                images.append(image_resized)
#                 b = "Leyendo..." + str(cant)
    return images,cant
# Asegúrate de reemplazar 'ruta_de_salida' con la ubicación donde deseas guardar las imágenes aumentadas para cada categoría
# ruta_de_salida = destino1
ruta_de_salida = origen


# Crear una instancia de ImageDataGenerator para el aumento de datos
datagen = ImageDataGenerator(
    rotation_range=40,          # Rango de rotación aleatoria en grados
    width_shift_range=0.2,      # Desplazamiento horizontal aleatorio
    height_shift_range=0.2,     # Desplazamiento vertical aleatorio
    shear_range=0.2,            # Distorsión de corte
    zoom_range=0.2,             # Zoom aleatorio
    horizontal_flip=True,       # Volteo horizontal aleatorio
    fill_mode='nearest'         # Modo de relleno
)


maderasCat = ['Ficus_sp','Ocotea_bofo','Qualea_acuminata','Tabebuia_rosea',
'Cariniana_pyriformis','Tectona_grandis','Pithecellobium_dulce',
'Peltogyne_purpurea','Anacardium_excelsum','Mangifera_indica',
'Guazuma_ulmifolia','Pinus_radiata','Quercus_humboldtii','Cordia_alliodora',
'Gmelina_arborea','Albizia_guachapele','Cedrelinga_cateniformis',
'Erythrina_sp','Clathrotropis_brunnea','Pinus_patula','Eucalyptus_tereticornis',
'Couma_macrocarpa','Centrolobium_yavizanum','Xylosma_benthamii',
'Melicoccus_bijugatu','Jacaranda_copaia','Cedrela_odorata','Campnosperma_panamensis',
'Ocotea_insularis']

num_categorias = len(maderasCat) # variable para contar las categorias


# Inicializa un contador para el total de imágenes aumentadas
total_imagenes_aumentadas = 0

# Dentro del bucle que aumenta las imágenes para cada categoría
for categoria in maderasCat:
    images_per_category, cant = cargar_imagenes_de_categoria(categoria)  # Carga las imágenes de la categoría
    total_images_in_category = len(images_per_category)

    # Inicializa un contador para las imágenes aumentadas en la categoría actual
    total_images_aumentadas_in_category = 0

    # Mientras el número de imágenes en la categoría sea menor que 10 (no 20)
    while total_images_in_category <= 10:
        # Genera imágenes aumentadas y agrégalas a la categoría
        augmented_images = []
        for image in images_per_category:
            # Expande las dimensiones del tensor de imagen
            image_expanded = np.expand_dims(image, axis=0)
            for batch in datagen.flow(image_expanded, batch_size=1):
                augmented_images.append(batch[0])
                total_images_aumentadas_in_category += 1  # Incrementa el contador de imágenes aumentadas
                break  # Solo necesitamos generar 1 imagen aumentada por vez
            if total_images_aumentadas_in_category >= 10:
                break

        images_per_category.extend(augmented_images)
        total_images_in_category = len(images_per_category)

    # Restaura la forma original de las imágenes
    images_per_category = np.array(images_per_category)

    # Guarda las imágenes aumentadas en la categoría correspondiente
    categoria_dir = os.path.join(data_dir, categoria)
    for i, augmented_image in enumerate(images_per_category[len(images_per_category) - len(augmented_images):]):
        filename = f'{i + 1}_augmented.jpg'
        filepath = os.path.join(categoria_dir, filename)
        plt.imsave(filepath, augmented_image.astype(np.uint8))


    # Suma la cantidad de imágenes aumentadas en la categoría actual al total de imágenes aumentadas
    total_imagenes_aumentadas += total_images_aumentadas_in_category
    
    # suma del array
    dircount.append(total_images_aumentadas_in_category)
    
    #guardado de directorios y/o nombre de las categorias
    directories.append(categoria)
    
    
    # Imprime la cantidad de imágenes en la categoría actual
    print(f'en la Categoría: {categoria}, el total de imagenes es: {total_images_aumentadas_in_category}')
            

    
# Imprime el número total de categorías y la suma total de imágenes aumentadas en todas las categorías
print(f'Número total de categorías: {num_categorias}')
print(f'Suma Total de imágenes aumentadas en todas las categorías: {total_imagenes_aumentadas}')
print(f'array {dircount}')

leyendo imagenes de  /home/jorge/EjemplosPY/WoodTracer/Wood60X/imgs60X/
en la Categoría: Ficus_sp, el total de imagenes es: 10
en la Categoría: Ocotea_bofo, el total de imagenes es: 10
en la Categoría: Qualea_acuminata, el total de imagenes es: 10
en la Categoría: Tabebuia_rosea, el total de imagenes es: 10
en la Categoría: Cariniana_pyriformis, el total de imagenes es: 10
en la Categoría: Tectona_grandis, el total de imagenes es: 10
en la Categoría: Pithecellobium_dulce, el total de imagenes es: 10
en la Categoría: Peltogyne_purpurea, el total de imagenes es: 10
en la Categoría: Anacardium_excelsum, el total de imagenes es: 10
en la Categoría: Mangifera_indica, el total de imagenes es: 10
en la Categoría: Guazuma_ulmifolia, el total de imagenes es: 10
en la Categoría: Pinus_radiata, el total de imagenes es: 10
en la Categoría: Quercus_humboldtii, el total de imagenes es: 10
en la Categoría: Cordia_alliodora, el total de imagenes es: 10
en la Categoría: Gmelina_arborea, el total de ima

## Creamos las etiquetas

In [9]:
labels=[]
indice=0
for cantidad in dircount:
    for i in range(cantidad):
        labels.append(indice)
    indice=indice+1
print("Cantidad etiquetas creadas: ",len(labels))

Cantidad etiquetas creadas:  290


In [10]:
maderas=[]
indice=0
for directorio in directories:
    name = directorio.split(os.sep)
    print(indice , name[len(name)-1])
    maderas.append(name[len(name)-1])
    indice=indice+1

0 Ficus_sp
1 Ocotea_bofo
2 Qualea_acuminata
3 Tabebuia_rosea
4 Cariniana_pyriformis
5 Tectona_grandis
6 Pithecellobium_dulce
7 Peltogyne_purpurea
8 Anacardium_excelsum
9 Mangifera_indica
10 Guazuma_ulmifolia
11 Pinus_radiata
12 Quercus_humboldtii
13 Cordia_alliodora
14 Gmelina_arborea
15 Albizia_guachapele
16 Cedrelinga_cateniformis
17 Erythrina_sp
18 Clathrotropis_brunnea
19 Pinus_patula
20 Eucalyptus_tereticornis
21 Couma_macrocarpa
22 Centrolobium_yavizanum
23 Xylosma_benthamii
24 Melicoccus_bijugatu
25 Jacaranda_copaia
26 Cedrela_odorata
27 Campnosperma_panamensis
28 Ocotea_insularis


In [11]:
y = np.array(labels)
X = np.array(images, dtype=np.uint8) #convierto de lista a numpy

# Find the unique numbers from the train labels
classes = np.unique(y)
nClasses = len(classes)
print('Total number of outputs : ', nClasses)
print('Output classes : ', classes)

Total number of outputs :  29
Output classes :  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28]


## Creamos Sets de Entrenamiento y Test

In [12]:
train_X,test_X,train_Y,test_Y = train_test_split(X,y,test_size=0.1)
print('Training data shape : ', train_X.shape, train_Y.shape)
print('Testing data shape : ', test_X.shape, test_Y.shape)

ValueError: Found input variables with inconsistent numbers of samples: [0, 290]

In [ ]:
plt.figure(figsize=[5,5])

# Display the first image in training data
plt.subplot(121)
plt.imshow(train_X[0,:,:], cmap='gray')
plt.title("Ground Truth : {}".format(train_Y[0]))

# Display the first image in testing data
plt.subplot(122)
plt.imshow(test_X[0,:,:], cmap='gray')
plt.title("Ground Truth : {}".format(test_Y[0]))

## Preprocesamos las imagenes

In [ ]:
train_X = train_X.astype('float32')
test_X = test_X.astype('float32')
train_X = train_X / 255.
test_X = test_X / 255.

## Hacemos el One-hot Encoding para la red

In [ ]:
# Change the labels from categorical to one-hot encoding
train_Y_one_hot = to_categorical(train_Y)
test_Y_one_hot = to_categorical(test_Y)

# Display the change for category label using one-hot encoding
print('Original label:', train_Y[0])
print('After conversion to one-hot:', train_Y_one_hot[0])

## Creamos el Set de Entrenamiento y Validación

In [ ]:
# Mezclar todo y crear los grupos de entrenamiento y testing
train_X,valid_X,train_label,valid_label = train_test_split(train_X, train_Y_one_hot, test_size=0.2, random_state=13)

In [ ]:
print(train_X.shape,valid_X.shape,train_label.shape,valid_label.shape)

## Creamos el modelo de CNN

In [ ]:
#declaramos variables con los parámetros de configuración de la red
INIT_LR = 1e-3 # Valor inicial de learning rate. El valor 1e-3 corresponde con 0.001
epochs = 400 # Cantidad de iteraciones completas al conjunto de imagenes de entrenamiento
batch_size = 64 # cantidad de imágenes que se toman a la vez en memoria

In [ ]:
# Create the model
wood_model = Sequential()

wood_model.add(Conv2D(32, kernel_size=(3, 3), activation='linear', padding='same', input_shape=(50,50,3)))
wood_model.add(Conv2D(64, kernel_size=(3, 3), activation='linear', padding='same'))
wood_model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
wood_model.add(Dropout(0.25))

wood_model.add(Conv2D(128, kernel_size=(3, 3), activation='linear', padding='same'))
wood_model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
wood_model.add(Conv2D(128, kernel_size=(3, 3), activation='linear', padding='same'))
wood_model.add(MaxPooling2D(pool_size=(2, 2)))
wood_model.add(Dropout(0.25))

wood_model.add(Flatten())
wood_model.add(Dense(1024, activation='linear'))
wood_model.add(Dropout(0.5))
#wood_model.add(Dense(nClasses, activation='softmax'))
wood_model.add(Dense(nClasses, activation='softmax'))

In [ ]:
wood_model.summary()

In [ ]:
plot_model(wood_model, to_file='model_plot3.png', show_shapes=True, show_layer_names=True)

In [ ]:
wood_model.compile(loss=keras.losses.categorical_crossentropy, optimizer=tf.keras.optimizers.Adagrad(lr=INIT_LR, decay=INIT_LR / 100),metrics=['accuracy'])

## Entrenamos el modelo: Aprende a clasificar imágenes

In [ ]:
# este paso puede tomar varios minutos, dependiendo de tu ordenador, cpu y memoria ram libre
wood_train = wood_model.fit(train_X, train_label, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(valid_X, valid_label))

In [ ]:
# guardamos la red, para reutilizarla en el futuro, sin tener que volver a entrenar
wood_model.save("Wood2GPU3_mnist.h5")

## Evaluamos la red

In [ ]:
test_eval = wood_model.evaluate(test_X, test_Y_one_hot, verbose=1)

In [ ]:
print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])

In [ ]:
accuracy = wood_train.history['accuracy']
val_accuracy = wood_train.history['val_accuracy']
loss = wood_train.history['loss']
val_loss = wood_train.history['val_loss']
epochs = range(len(accuracy))
plt.plot(epochs, accuracy, 'bo', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
predicted_classes2 = wood_model.predict(test_X)

In [ ]:
predicted_classes=[]
for predicted_wood in predicted_classes2:
    predicted_classes.append(predicted_wood.tolist().index(max(predicted_wood)))
predicted_classes=np.array(predicted_classes)

In [ ]:
predicted_classes.shape, test_Y.shape

## Aprendamos de los errores: Qué mejorar

In [ ]:
correct = np.where(predicted_classes==test_Y)[0]
print("Found %d correct labels" % len(correct))
for i, correct in enumerate(correct[0:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(test_X[correct].reshape(50,50,3), cmap='gray', interpolation='none')
    plt.title("{}, {}".format(maderas[predicted_classes[correct]],
                                                    maderas[test_Y[correct]]))

    plt.tight_layout()

In [ ]:
incorrect = np.where(predicted_classes!=test_Y)[0]
print("Found %d incorrect labels" % len(incorrect))
for i, incorrect in enumerate(incorrect[0:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(test_X[incorrect].reshape(50,50,3), cmap='gray', interpolation='none')
    plt.title("{}, {}".format(maderas[predicted_classes[incorrect]],
                                                    maderas[test_Y[incorrect]]))
    plt.tight_layout()

In [ ]:
target_names = ["Class {}".format(i) for i in range(nClasses)]
print(classification_report(test_Y, predicted_classes, target_names=target_names))

## Prediccion de una nueva imagen¶

In [ ]:
dirname = os.path.join(os.getcwd(), 'val60X')
imgpath = dirname + os.sep 

images2 = []
filenames1 = []
cont = 0

print("leyendo imagenes de ",imgpath)

for root, dirnames, filenames in os.walk(imgpath):
    for filename in filenames:
        if re.search("\.(jpg|jpeg|png|bmp|tiff)$", filename):
            filepath = os.path.join(root, filename)
            image = plt.imread(filepath)
            image_resized = resize(image, (50, 50),anti_aliasing=True,clip=False,preserve_range=True)
            images.append(image_resized)
        filenames1.append(filename)         
    X = np.array(images2, dtype=np.uint8) #convierto de lista a numpy
    test_X = X.astype('float32')
    test_X = test_X / 255.

predicted_classes = wood_model.predict(test_X)

for i, img_tagged in enumerate(predicted_classes):
    cadena = filenames1[i]
    if (cadena.count(maderas[img_tagged.tolist().index(max(img_tagged))])):
        cont = cont + 1
    print(filenames1[i],' -> ',maderas[img_tagged.tolist().index(max(img_tagged))])
    
print("______________________________________________________________")
    
print("Probabilidad de Deteccion: {:.2f}%".format((cont/(i+1)*100)))
    

In [ ]:
fin = time.time()
print(fin-inicio) 


In [ ]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()

In [ ]:
!mkdir carpeta_salida-7092023

In [ ]:
!tensorflowjs_converter --input_format keras Wood2GPU3_mnist-7092023.h5 carpeta_salida-7092023